# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Таким образом, делаем вывод о том, что **цель исследования** — проверка гипотезы:
1. Разные цели кредита влияют на факт погашения кредита в срок.
2. Уровень дохода клиента влияет на факт погашения кредита в срок.
3. Семейное положение клиента вличяет на факт погашения кредита в срок.
4. Количество детей клиента влияет на факт погашения кредита в срок.


## Шаг 1. Откроем файл с данными и изучим общую информацию

Для работы с гипотезами необходима работа с данными. Для работы же сданными нам необходимо с помощью `библиотеки` pandas прочитать датасет.

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Итак, в таблице двенадцать столбцов. Тип данных в столбцах различается.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — образование клиента;
* `education_id` — идентификатор образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — доход в месяц;
* `purpose` — цель получения кредита.

В названиях колонок нет нарушения стиля.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


В каждой строке таблицы — данные о клиентах. Каждая из колонок содержит определенные данные о клиентах. Одна строка - одни клиент и, соответственно, данные о нем. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.
Возможно наличие дубликатов, связанное с использованием различных регистров в столбце `education`, а также тем, что в столбце `days_employed`представлены как отрицательные, так и положительные числа.
В таблице встречаются вещественные числа - для удобства работы с данными считается разумным перевести их в целочисленные.
В столбце `children` заметны ошибки - максимум детей: 20, а минимум: -1.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Чтобы можно было работать с данными, необходимо сначала посчитать пропуски, а затем устранить их или заменить по возможности.

In [6]:
#подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим, что пропуски присутствуют только в двух столбцах - `days_employed` и `total_income`. Если трудовой стаж для анализа нашей гипотезы не играет серьезной роли, то уровень дохода необходим для подверждения гипотезы. Пропуски в столбце со стажем могли возникнуть по причине отсутствия трудового стажа у клиентов, которые просто не стали заполнять данную графу. 
Пропуски же в столбце с доходом, вероятно, возникли по причине ошибки самих клиентов - возможно они забыли указать их. Также возможна техническая ошибка.

Проверим предположение по поводу столбца `days_employed` и посчитаем количество клиентов с трудовы стажем равным 0.

In [7]:
#Считаем количество занчений в столбце, равных 0
df[df['days_employed'] == 0]['days_employed'].count()

0

Предположение подтвердилось. Вероятнее всего, клиенты не имеющие трудовой стаж просто не заполняли подобные данные о себе.
Поскольку пропуски в столбце `days_employed` не влияют на исследование, то можно заменить все пропуски в данном столбце на 0.

In [8]:
#Заменяем все пропуски в столбце на 0
df.loc[df['days_employed'].isna(), 'days_employed'] = 0

Для удобства работы с данными и эстетического вида таблицы изменим все значения в столбце `days_employed` с дней на года. А затем все аномально большие значения заменим на более приемлемые.

In [9]:
#переименовываем название столбца
df = df.rename(columns={'days_employed':'years_employed'})

In [10]:
#переводим дни в года (без учета високосных лет)
df['years_employed'] = df['years_employed']/365

In [11]:
#проверяем полученный результат
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В глаза сразу бросается аномально большое значение - нужно от него избавиться. Возможно, клиенты банка указывали стаж не в днях, а в часах - тогда это объясняет возникновение подобных аномалий.

Создадим цикл, который будет избавляться от аномально больших значений, деля их на 24 (ровно столько часов в сутках).

За аномально большие значения воззьмем те, которые превышает возраст клиентов.

In [12]:
#цикл для избавления от аномальных значений
for index in range(len(df['years_employed'])):
    if df['years_employed'][index] > df['dob_years'][index]:
        df['years_employed'][index] = df['years_employed'][index]/24
        import warnings
        warnings.filterwarnings("ignore")
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посчитаем количество пропусков в столбцу со стажем после замены

In [13]:
#Считаем количество пропусков после замены
df['years_employed'].isna().sum()

0

С пропусками в столбце `days_employed` мы разобрались - теперь необходимо устранить пропуски в столбце `total_income`.
Заменить пропуски в этом столбце можно медианой. Однако можно разумно полагать, что доход в месяц в зависимости от типа занятости может значительно отличаться.

Создадим цикл, который будет принимать уникальные значения столбца `income_type` и рассчитает медианный доход для каждого из типов занятости

In [14]:
#уникальные значения столбца income_type
income_type_uniq = df['income_type'].unique()

In [15]:
#рассчитаем медианный доход клиентов в зависимости от типа занятости с помощью цикла
for elem in income_type_uniq:
    salary = df.loc[df['income_type'] == elem, 'total_income' ].median()
    print(f'{elem} - {salary}')

сотрудник - 142594.39684740017
пенсионер - 118514.48641164352
компаньон - 172357.95096577113
госслужащий - 150447.9352830068
безработный - 131339.7516762103
предприниматель - 499163.1449470857
студент - 98201.62531401133
в декрете - 53829.13072905995


Предположения подтвердились, и в зависимости от типа занятости доходы клиентов значительно различаются. Поэтому разумным будет сгруппировать данные по столбцу `income_type` и найти медианный доход для каждой из групп. Затем заменить все пропуски на полученные данные в зависимости от группы.

In [16]:
#заменяем пропуски в столбце income_type на медианной значение
for elem in income_type_uniq:
    median = df.loc[df['income_type'] == elem, 'total_income' ].median()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == elem), 'total_income'] = median

Посчитаем количество пропусков в столбцу со стажем после замены

In [17]:
#Считаем количество пропусков после замены
df['total_income'].isna().sum()

0

Для удобства работы с данными разделим значения в столбце с ежемесячным доходом на 1000, чтобы значения измерялись в тысячах. 
А затем посчитаем количество дубликатов.

In [18]:
#переводим значения столбца в тысячи
df['total_income'] = df['total_income'] /1000

In [19]:
#считаем дубликаты
df['years_employed'].isna().sum()

0

In [20]:
#переименум столбце
df = df.rename(columns={'total_income':'total_income(*1000)'})

In [21]:
#смотрим на внешний вид таблицы
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose
0,1,-23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875639,покупка жилья
1,1,-11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080014,приобретение автомобиля
2,0,-15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145.885952,покупка жилья
3,3,-11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628550,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616078,сыграть свадьбу


**Вывод**

Таким образом, были заполены все пропущенные значения в столбцах `income_type` и `days_employed`.
Поскольку столбец `days_employed` не важен для исследования, то пропущенные значения в нем были заменены на 0 с помомощью <b>loc</b>.
Стобец же `income_type` необходим для подтверждения одной из гипотез, поэтому пропуски в нем были заменены на медианные значения в зависимости от типа занятости с использованием <b>loc</b>.

### Замена типа данных

Данные в столбцах `total_income` и `days_employed` имеют тип <b>float64</b>.
Представление доходов клиентов и дней стажа клиентов в вещественном типе данных кажется лишним и препятствующем обобщению результатов. Поэтому лучше заменить в данных столбцах тип данных на <b>целочисленный</b>.

In [22]:
#заменяем тип данных в столбцах на целочисленный
df[['total_income(*1000)', 'years_employed']] = df[['total_income(*1000)', 'years_employed']].astype('int')

In [23]:
#получаем общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children               21525 non-null int64
years_employed         21525 non-null int64
dob_years              21525 non-null int64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income(*1000)    21525 non-null int64
purpose                21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы заменили типы данных в столбцах `total_income` и `days_employed` на <b>int64</b> с помощью метода astype(). Теперь работать с данными в данных столбцах стало намного проще и удобнее. Был использован именно метод `astype()`, а не метод `to_numeric`, поскольку второй метод может заменять тип данных только на <b>float64</b>, а нам нужен был именно <b>int64</b>.

### Обработка дубликатов

Теперь необходимо обработать дубликаты. Для начала посчитаем явные дубликаты в таблице

In [24]:
# подсчёт явных дубликатов
df.duplicated().sum()

55

Вызываем специальный метод pandas, чтобы удалить явные дубликаты

In [25]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Потворно считаем явные дубликаты в таблице

In [26]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Теперь нужно избавиться от неявных дубликатов.

Для начала приведем столбец `education`в порядок. При выводе на экран первых 10 строк таблицы мы увидели, что в данном столбце значения различаются по регистру. Поэтому для поиска неявных дубликатов приведем все значения в данном столбце к нижнему регистру с помощью метода str.lower().

In [27]:
#приводим значения в столбце education к нижнему регистру
df['education'] = df['education'].str.lower()

Теперь подсчитаем количество дубликатов

In [28]:
# подсчёт явных дубликатов
df.duplicated().sum()

17

Вызываем специальный метод pandas, чтобы удалить явные дубликаты. Затем подсчитываем заново количество дубликатов.

In [29]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

In [30]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

В столбце `days_employed` представлены как положительные, так и отрицательные значения. Поэтому приведем все значения в данном столбце к модулю.

In [31]:
#приводим значение столбца days_employed к модулю
df['years_employed'] = df['years_employed'].apply(abs)

Теперь проверим количество дубликатов после приведения значений к модулю

In [32]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Дубликаты после приведения в порядок не были обнаружены. Замечены некоторые выбивающиеся значения, которые не могут соответсвовать действительности - в частности, у некоторых клиентов стаж превышает возраст, но так, как данные столбцы не играют роли в нашем исследовании, трогать мы их не будем.

**Вывод**

Мы обнаружили в таблице сначала явные дубликаты сочетанием методов **`duplicated()`** и **`sum()`**, а затем удалили их методом **`drop_duplicates()`**. Также мы выровняли индексацию конструкцией **`.reset_index(drop=True)`** Явные дубликаты появились в таблице вероятнее всего из-за технической ошибки или невнимательности человека.
Затем привели значения в столбце `days_employed` к модулю с помощью **`abs()`** и значения в столбце `education` к нижнему регистру с помощью **`str.lower()`**, чтобы найти неявные дубликаты, а затем удалили их. Полагается, что неявные дубликаты возникли в результате того, что один и тот же клиент был посчитан два в связи с тем, что у них различались регистры. Вероятно, это система посчитала их за двух разных людей.


### Лемматизация

Для подтверждаения первой гипотезы, говорящей о связи между целью получения кредита и его возвратом в срок, необходимо будет произвести категоризацию данных по цели получения кредита. Для этого выведим список уникальных значений `purpose`, чтобы узнать, какие цели присутствуют в датасете. Если будут обнаружены схожие цели, то нужно будет привести их к единству.

In [33]:
#подсчет уникальных значений
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              504
заняться высшим образованием      

Результаты показывают, что многие клиенты брали кредит для одинаковых целей при этом, указывая их названия по-разному. Для подвтерждения первой гипотезы нужно будет провести категоризацию по столбцу `purpose`, но для этого нужно приветси цели к единообразию - для этого нужна лемматизация.

In [34]:
#импортируем модуль pymystem3
from pymystem3 import Mystem
m = Mystem()
#илемматизируем столбец purpose
df['category_purpose'] = df['purpose'].apply(m.lemmatize)
#считаем уникальные значения после лемматизации
df['category_purpose'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

**Вывод**

Мы получили столбец, который содержит списки с отлемматизированными целями получения кредита. Данный столбец мы получили с помощью использования метода `apply()`, который позволил применить конструкцию `Mystem().lemmatize` к каждой строке датафрейма, не используя при этом цикл. Затем мы вывели уникальные значения данного столбца, чтобы использовать их в дальнейшей категоризации. Среди уникальных значений в списках мы уже можем найти основные цели, по которым мы и будем категоризировать данные.

Так, все отлемматизированные списки содержат хотя бы одну из этих целей - *образование, свадьба, автомобиль, недвижимость, жилье.*

### Категоризация данных

Для того, чтобы подвтердить или опровергнуть гипотезу, нужно совершить категоризацию данных. Она поможет нам проследить взаимосвязь определенных категорий.

#### Категоризация данных для работы с первой гипотезой (цель кредита)

Создадим функцию, которая отфильтрует цели получения кредита на категории. Выделим 4 категории:
1) свадьба 

2) автомобиль

3) образование

4) недвижимость (объединяет понятие жилья и недвижимости)

In [35]:
#создаем функцию для выеделения категорий
def filter(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'образование' in row:
        return 'образование'
    return 'недвижимость'
   

Функция применяет в качестве аргумента столбец, а выводим категорию в зависимости от цели, которая указана в столбце.

In [36]:
#применяем функцию с помощью метода apply и сохраняем ее результат в новый столбец category_purpose
df['category_purpose'] = df['category_purpose'].apply(filter)
#выводим на экран последние 10 строк таблицы
df.tail(10) 

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose,category_purpose
21443,1,1,28,среднее,1,женат / замужем,0,F,сотрудник,1,109,заняться образованием,образование
21444,0,2,42,высшее,0,женат / замужем,0,F,компаньон,0,322,покупка своего жилья,недвижимость
21445,0,1,42,высшее,0,гражданский брак,1,F,компаньон,0,178,на покупку своего автомобиля,автомобиль
21446,0,42,59,среднее,1,женат / замужем,0,F,пенсионер,0,153,сделка с автомобилем,автомобиль
21447,1,6,37,ученая степень,4,в разводе,3,M,сотрудник,0,115,покупка коммерческой недвижимости,недвижимость
21448,1,12,43,среднее,1,гражданский брак,1,F,компаньон,0,224,операции с жильем,недвижимость
21449,0,39,67,среднее,1,женат / замужем,0,F,пенсионер,0,155,сделка с автомобилем,автомобиль
21450,1,5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89,недвижимость,недвижимость
21451,3,8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244,на покупку своего автомобиля,автомобиль
21452,2,5,40,среднее,1,женат / замужем,0,F,сотрудник,0,82,на покупку автомобиля,автомобиль


На первый взгляд все выглядит ровно, но нужно проверить, так ли это на самом деле. Выведем уникальные значения столбца `category` на экран.

In [37]:
#выводим уникальные значения столбца с категориями
df['category_purpose'].value_counts()

недвижимость    10811
автомобиль       4305
образование      4013
свадьба          2324
Name: category_purpose, dtype: int64

Функция справилась со своей задачей - мы получили 4 категории целей, с которыми уже можно работать.

#### Категоризация данных для работы со второй гипотезой (уровень дохода)

Теперь для работы со второй гипотезой, говорящей о связи уровня дохода клиента и возвратом кредита в срок, нужна категоризация по доходам клиента. Но для начала мы должны будем оценить диапазон получаемых доходов.

In [38]:
#отсортируем доход клиентов в порядке убывания
df['total_income(*1000)'].sort_values(ascending = False)

12389    2265
19547    2200
9158     1726
20741    1715
17136    1711
         ... 
14246      21
1598       21
12982      21
16137      21
14554      20
Name: total_income(*1000), Length: 21453, dtype: int64

Из результатов видно, что доход колеблется в пределах от 2 тысяч до 2 миллионов. Для того, чтобы выделить категории, необходимо найти медианное значение, чтобы опираясь на него, выделять категории.

In [39]:
#ищем медиану
df['total_income(*1000)'].median()

142.0

Медиана равна 142 тысячи. Она должна будет войти в группу, которая будет являться серединой. Необходимо распределить доходы клиентов на более-менее равные группы. Поделим клиентов на 3 равные групп, где:
1. *низкий доход* - до 125.000
2. *средний доход* - от 125.000 до 180.000 (в середину вошла медиана)
3. *высокий доход* -  все остальное, а именно свыше 180.000 


In [40]:
#строим собственную функцию для категоризации
def income(row):
    if row < 125:
        return 'низкий'
    
    if 125 < row < 180:
        return 'средний'
    
    return 'высокий'
#применяем собственную функцию к столбцу с доходами, а результат помещаем в новый столбец
df['category_income'] = df['total_income(*1000)'].apply(income)    
#выводим на экран уникальные значения полученных категорий
df['category_income'].value_counts()

низкий     7755
средний    7179
высокий    6519
Name: category_income, dtype: int64

Мы получили 3 более-менее равные для анализа категории. Собственная функция принимает в качестве аргумента столбец, и сверяет его данные с теми, что указаны внутри самой функции, при проверке же она присваивает категорию каждому клиенту в зависимости от его дохода. Проверим, как теперь выглядит таблица.

In [41]:
#выводим на экран первые 10 строк таблицы
df.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose,category_purpose,category_income
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья,недвижимость,высокий
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля,автомобиль,низкий
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья,недвижимость,средний
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование,образование,высокий
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу,свадьба,средний
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255,покупка жилья,недвижимость,высокий
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240,операции с жильем,недвижимость,высокий
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135,образование,образование,средний
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95,на проведение свадьбы,свадьба,низкий
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144,покупка жилья для семьи,недвижимость,средний


#### Категоризация данных для работы с третьей гипотезой (семейное положение)

Для исследования 3 гипотезы нужно будет разобраться, как соотносятся между собой столбцы `family_status` и `family_status_id`. В этом нам поможет создание словаря.

Для начала мы выведем уникальные значения столбца `family_status` для дальнейшего создания словаря.

In [42]:
#выводим уникальные значения столбца family_status
df['family_status'].value_counts()

женат / замужем          12338
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Для красоты картинки приведем данные к одному регистру методом `.str.lower()` 

In [43]:
#приводим значения столбца к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

Теперь выведем на экран уникальные значения

In [44]:
#выводим уникальные значения столбца family_status
df['family_status'].value_counts()

женат / замужем          12338
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Мы видим, что клиенты делятся на 5 групп. Теперь разделим исходный датафрейм на 2 таблицы - 1-ая будет словарем, а 2-ая хранить остальную.

In [45]:
#создаем таблицу с информацией о клиентах и их задолжностях
df1 = df[[ 'family_status_id', 'debt', 'children', 'years_employed', 'dob_years', 'education', 'education_id', 'family_status_id', 'gender', 'income_type', 'total_income(*1000)', 'purpose','category_purpose','category_income']]
#выводим на экран первые 10 строк полученной таблицы
df1.head(10)

,family_status_id,debt,children,years_employed,dob_years,education,education_id,family_status_id,gender,income_type,total_income(*1000),purpose,category_purpose,category_income
0,0,0,1,23,42,высшее,0,0,F,сотрудник,253,покупка жилья,недвижимость,высокий
1,0,0,1,11,36,среднее,1,0,F,сотрудник,112,приобретение автомобиля,автомобиль,низкий
2,0,0,0,15,33,среднее,1,0,M,сотрудник,145,покупка жилья,недвижимость,средний
3,0,0,3,11,32,среднее,1,0,M,сотрудник,267,дополнительное образование,образование,высокий
4,1,0,0,38,53,среднее,1,1,F,пенсионер,158,сыграть свадьбу,свадьба,средний
5,1,0,0,2,27,высшее,0,1,M,компаньон,255,покупка жилья,недвижимость,высокий
6,0,0,0,7,43,высшее,0,0,F,компаньон,240,операции с жильем,недвижимость,высокий
7,0,0,0,0,50,среднее,1,0,M,сотрудник,135,образование,образование,средний
8,1,0,2,18,35,высшее,0,1,F,сотрудник,95,на проведение свадьбы,свадьба,низкий
9,0,0,0,5,41,среднее,1,0,M,сотрудник,144,покупка жилья для семьи,недвижимость,средний


In [46]:
#создаем таблицу-словарь
df2 = df[['family_status', 'family_status_id']]
#выводим на экран первые 10 строк полученной таблицы
df2.head(10)

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
5,гражданский брак,1
6,женат / замужем,0
7,женат / замужем,0
8,гражданский брак,1
9,женат / замужем,0


В таблице с категоризацией данных (словаре) имеются дубликаты - некоторые типы категорийповторяются несколько раз. Поэтому необходимо удалить дубликаты.

In [47]:
#удаляем дубликаты
df2 = df2.drop_duplicates().reset_index(drop = True).sort_values(by = 'family_status_id')
#выводим полученную таблицу на экран
df2

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Теперь мы видим, как соотносятся идентификатор семейного положения и само семейное положение.

#### Категоризация данных для работы с четвертой гипотезой (количество детей)

Для решения четвертой гипотезы нужно найти уникальные значения столбца с количеством детей

In [48]:
#выводим уникальные значения столбца
df['children'].value_counts()

 0     14091
 1      4808
 2      2051
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Результаты оказались довольно удивительными. Все бы ничего, но в таблице присутствуют два выбивающихся значения. Этими значениями являются 20 детей и -1 ребенок. Кажется вполне очевидным, что подобные результаты возникли в связи с человеческим фактором, а именно по вине клиентов. Видимо, когда они указывали свои данные вместо 1 ребенка указали по ошибке -1, а когда указывали 2 детей - по ошибке указали 20.
Нужно проверить, как сильно изменятся данные в случае, если мы заменим 20 на 2, а -1 на 1

Для начала мы извлечем информацию об уплате кредита в срок, сгруппируем ее по количеству детей, посчитаем сколько клиентов каждой группы так оличились, а затем все отсортируем.

In [49]:
#группируем данные с клиентами, просрочившими платеж по количеству у них детей
df[df['debt'] == 0].groupby('children')['children'].count().sort_values()


children
 5         9
 4        37
-1        46
 20       68
 3       303
 2      1857
 1      4364
 0     13028
Name: children, dtype: int64

Затем сделаем все то же самое, но для тех, кто просрочил уплату кредита

In [50]:
df[df['debt'] == 1].groupby('children')['children'].count().sort_values()

children
-1        1
 4        4
 20       8
 3       27
 2      194
 1      444
 0     1063
Name: children, dtype: int64

Как видно из результатов, если мы добавим клиентов, которые указали, что у них - 1 ребенок к клиентам, у которых 1 ребенок, и 20 детей к тем, у которых 2 ребенка, то результат исследования не исказится.

In [51]:
#приводим значение к модулю, чтобы избавиться от -1
df['children'] = df['children'].apply(abs)
#создаем собственную функцию для того, чтобы 20 детей превратить в двух
def child2(row):
    if row == 20:
        return int(2)
    return row
#применяем функцию к столбцу children
df['children']  = df['children'].apply(child2)
#выводим на экран первые 10 строк таблицы
df.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose,category_purpose,category_income
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253,покупка жилья,недвижимость,высокий
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112,приобретение автомобиля,автомобиль,низкий
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145,покупка жилья,недвижимость,средний
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267,дополнительное образование,образование,высокий
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158,сыграть свадьбу,свадьба,средний
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255,покупка жилья,недвижимость,высокий
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240,операции с жильем,недвижимость,высокий
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135,образование,образование,средний
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95,на проведение свадьбы,свадьба,низкий
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144,покупка жилья для семьи,недвижимость,средний


Проверим результат выполнения операции, выведя уникальные значения столбца

In [52]:
#выводим уникальные значения столбца
df['children'].value_counts()

0    14091
1     4855
2     2127
3      330
4       41
5        9
Name: children, dtype: int64

Теперь данные отображены корректно.

**Вывод**

Мы провели категоризацию данных для подвтверждения гипотез:
1) Чтобы узнать, есть ли связь между различными целями кредита и фактом погашения кредита в срок, мы прокатегоризировали данные из столбца с целями получения кредита с помощью собственной функции, опирающейся на результаты лемматизации - получили 4 категории целей.

2) Чтобы узнать, есть ли связь между уровнем дохода клиента влияет и фактом погашения кредита в срок, мы создали специальную функцию, которая поделила клиентов в зависимости от их дохода на 3 равные группы.

3) Чтобы узнать, есть ли связь между семейным положением клиента и фактом погашения кредита в срок, мы создали таблицу-словарь, которая хранит категории семейного положения - получили 5 категорий.

4) Чтобы узнать, есть ли связь между количеством детей у клиента и фактом погашения кредита в срок, мы нашли уникальные значения столбца с количеством детей, которые послужат категориями - получилось 6 категорий.

После категоризации можно приступать непосредственно к анализу гипотез

## Шаг 3. Ответим на вопросы

### Как разные цели кредита влияют на его возврат в срок?

Для начала выведем полученные категории целей

In [53]:
#выводим уникальные значения
df['category_purpose'].value_counts()

недвижимость    10811
автомобиль       4305
образование      4013
свадьба          2324
Name: category_purpose, dtype: int64

Для работы с данными создадим сводную таблицу с помощью pivot_table, где:
- *index* - категория цели (`category_purpose`)
- *columns* - неуплата вовремя кредита (`debt`)
- *values* - значение может быть любое, поскольку количество ячеек в столбцах одинаково. (я взял за основу - `education_id`)
- *aggfunc* - функция подсчета количества (`count`)

In [54]:
#создаем сводную таблицу
df_pivot = df.pivot_table(index ='category_purpose', columns = 'debt', values = 'education_id', aggfunc = 'count')
#выводим сводную таблицу
df_pivot

debt,0,1
category_purpose,,
автомобиль,3902,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


Для более детального анализа добавим в таблицу данные о том, какой процент составляют клиенты каждой категории от их сммурного количества. Для это сложим значения столбцев **1** и **0** и применим к ним метод `sum()`

In [55]:
#считаем бщее количество клиентов
tot_pay = (df_pivot[1]+df_pivot[0]).sum()
#выводим на экран общее количество клиентов
tot_pay

21453

Теперь рассчитаем, какой же процент составляют клиенты каждой категории. Добавим результат в новый столбец `pivot`. Таким образом, в `pivot` хранится процент, который составляют клиенты определенной категории от общего числа клиентов

In [56]:
#считаем процент клиентов и добавляем в отдельный столбец
df_pivot['pivot'] = ((df_pivot[1]+df_pivot[0])/tot_pay)*100
#выводим таблицу на экран
df_pivot

debt,0,1,pivot
category_purpose,,,
автомобиль,3902,403,20.067123
недвижимость,10029,782,50.393884
образование,3643,370,18.706008
свадьба,2138,186,10.832984


Мы получили данные для предварительного анализа. Так, мы видим, что чаще всего берут кредиты на **недвижимость**, следом за ней идет цель в виде покупки **автомобиля**, затем креди на **образование** и наконец кредит на **свадьбу**, который является наименне популярным среди представленных.
По количество оформленных кредитов можно составить такой рейтинг:
1. *Недвижимость* - чуть более 50%
2. *Автомобиль* - 20%
3. *Образование* - 18%
4. *Свадьба* - более 10%

Теперь добавим столбец, который содержит информацию о том, какой процент среди всех клиентов данной категории имел задолженность по кредиту, и назовем этот столбец `non-payment`. Также для более обширного анализа заодно и добавим процент тех, кто не имел задолженности и поместим результат в столбец `payment`.

In [57]:
#рассчитываем процент задолжником
df_pivot['non-payment'] = (df_pivot[1]/(df_pivot[1]+df_pivot[0]))*100
#рассчитываем процент не задолжников
df_pivot['payment'] = (df_pivot[0]/(df_pivot[1]+df_pivot[0]))*100
#выводим полученную таблицу на экран
df_pivot

debt,0,1,pivot,non-payment,payment
category_purpose,,,,,
автомобиль,3902,403,20.067123,9.361208,90.638792
недвижимость,10029,782,50.393884,7.233373,92.766627
образование,3643,370,18.706008,9.220035,90.779965
свадьба,2138,186,10.832984,8.003442,91.996558


Теперь исходя из это таблицы можем делать выводы о надежности клиентов каждой категории

**Вывод**

В целом процент невозврата кредита в срок  не сильно изменяется в зависимости от целей и колеблется в пределах от 7 до 9%. Однако проценты все-таки разные, поэтому подытожим результаты данного исследования.

В целом гипотеза подтвердилась, и разные цели получения кредита так или инчае влияют на факт возврата кредита в срок.

В частности, наблюдается интересная закономерность. Наибольшая группа клиентов, которая составляет более 50% от всех клиентов, группа клиентов, которая брала кредит на получение `недвижимости`, оказалась наиболее надежной. Среди всех клиентов, которые брали кредит на недвижимость, всего **7%** из них имели задолженность.

Вторая же по численности группа клиентов, бравших кредит на `автомобиль`, оказалась, наоборот, самой ненадежной. Именно клиенты данной группы чаще остальных просрачивали платеж кредита. Процент невозврата кредита в срок среди этой группы составляет практически 1/10 (**9,4%**) ото всех взявших кредит, что довольно велико.

Третья по численности группа клиентов, которая брала кредит на `образование` оказалась и третьей по надежности, поскольку ее процент невозврата кредита в срок составляет **9,22%**, что в целом равно результату, который показала вторая группа, любителей автомобилей. Можно сказать, что практически каждый 10 из второй и третей по численности групп не возвращает кредит в срок.

И наконец четвертая по численности группа клиентов, которые брали кредит на `свадьбу`, наименьшая среди всех групп клиентов, занимает второе место по надежности. Процент просрочки среди клиентов данной группы составляет **8%**. Это, конечно, тоже не мало, но в сравнении с третьей и второй группой все-таки терпимо.

Так можно составить рейтинги надежности клиентов в зависимости от цели получения кредита по процентам просрочки платежей:
1. Кредит на недвижимость (7%)
2. Кредит на свадьбу (8%)
3. Кредит на образование (9,2%)
4. Кредит на автомобиль (9,4%)

Самые надежные клиенты берут кредит на недвижимость, а самые ненадежные - на автомобиль.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


Для начала выведем полученные категории дохода

In [58]:
#выводим уникальные значения
df['category_income'].value_counts()

низкий     7755
средний    7179
высокий    6519
Name: category_income, dtype: int64

Для работы с данными создадим сводную таблицу с помощью pivot_table, где:
- *index* - категория цели (`category_income`)
- *columns* - неуплата вовремя кредита (`debt`)
- *values* - значение может быть любое, поскольку количество ячеек в столбцах одинаково. (я взял за основу - `education_id`)
- *aggfunc* - функция подсчета количества (`count`)

In [59]:
#создаем сводную таблицу
df_pivot2 = df.pivot_table(index ='category_income', columns = 'debt', values = 'education_id', aggfunc = 'count')
#выводим полученную таблицу на экран
df_pivot2

debt,0,1
category_income,,
высокий,6032,487
низкий,7112,643
средний,6568,611


Теперь рассчитаем, какой же процент составляют клиенты каждой категории по аналогии с прошлой гипотезой. Добавим результат в новый столбец `pivot`. Таким образом, в `pivot` хранится процент, который составляют клиенты определенной категории от общего числа клиентов

In [60]:
#считаем процент клиентов и добавляем в отдельный столбец
df_pivot2['pivot'] = ((df_pivot2[1]+df_pivot2[0])/tot_pay)*100
#выводим таблицу на экран
df_pivot2

debt,0,1,pivot
category_income,,,
высокий,6032,487,30.387358
низкий,7112,643,36.148790
средний,6568,611,33.463851


Мы получили данные для предварительного анализа. Так, мы видим, что процент оформления кредита для каждой из групп разнится. Проследуется закономерность - с повышением доходов клиентов понижается процент случаев взятия кредитов. Так, чем выше доход, тем меньше кредитов берут люди, и наоборот - чем ниже доход, тем чаще люди берут кредиты.

Чаще всего берут кредиты клиенты с доходом **до 125.000**, затем идут лица с доходом **от 125.000 до 180.000** и наименее заинтересованные в получении кредита среди представленных лица, доход которых составляет **более 180.000**.

По количество оформленных кредитов можно составить такой рейтинг:
1. *доход до 125.000* - 36,1%
2. *доход от 125.000 до 180.000* - 33,9%
3. *доход от 180.000* - 29,9%




Теперь добавим столбец, который содержит информацию о том, какой процент среди всех клиентов данной категории имел задолженность по кредиту, и назовем этот столбец `non-payment`. Также для более обширного анализа заодно и добавим процент тех, кто не имел задолженности и поместим результат в столбец `payment`.

In [61]:
#рассчитываем процент задолжником
df_pivot2['non-payment'] = (df_pivot2[1]/(df_pivot2[1]+df_pivot2[0]))*100
#рассчитываем процент не задолжников
df_pivot2['payment'] = (df_pivot2[0]/(df_pivot2[1]+df_pivot2[0]))*100
#выводим полученную таблицу на экран
df_pivot2

debt,0,1,pivot,non-payment,payment
category_income,,,,,
высокий,6032,487,30.387358,7.470471,92.529529
низкий,7112,643,36.148790,8.291425,91.708575
средний,6568,611,33.463851,8.510935,91.489065


**Вывод**

В целом процент невозврата кредита в срок  не сильно изменяется в зависимости от доходов клиента и колеблется в пределах от 7 до 8%. В таблице нет выделяющихся на фоне остальных значений процента просрочки

Наименее надежными заемщиками являются клиенты со `средним доходом` (от 125.000 до 180.000). Данная категория граждан просрачивали платеж в **8,5%** случаев.

Наиболее надежными заемщиками являются клиенты с `высоким доходом` (свыше 180.000). Данная категория граждан просрачивала платеж в **7,4%** случаев.

Между ними находится категория граждан с `низкими доходами` (до 125.000). Данная группа составляет **8.3%** случаев.

В целом из сводной таблицы можно сдлеать вывод, что клиенты со средним и низким доходом имели одинаковое количество просроченных платежей (**8.3-8.5%**). И отчасти гипотеза опровергается. Однако клиенты с более высоким доходом имели меньшее количество случаев просрочки, которое отличается от лиц с низкими и средними доходами, и составляет **7,4%**.

Таким образом, отвечая на вопрос, есть ли зависимость между доходом клиента и возвратом кредита в срок, можно сказать, что она есть, но довольно небольшая, и прослеживается в основном с более высокими уровнями дохода. Лица с наиболее большими доходами, значительно отличающимися от остальных, возвращают кредит в срок чаще, чем все остальные. 


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для начала выведем полученные категории дохода

In [62]:
#выводим уникальные значения
df['family_status'].value_counts()

женат / замужем          12338
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Для работы с данными создадим сводную таблицу с помощью pivot_table, где:
- *index* - категория цели (`family_status`)
- *columns* - неуплата вовремя кредита (`debt`)
- *values* - идентификатор семейного положения (`family_status_id`)
- *aggfunc* - функция подсчета количества (`count`)

In [63]:
#создаем сводную таблицу
df_pivot3 = df.pivot_table(index ='family_status', columns = 'debt', values = 'family_status_id', aggfunc = 'count')
#выводим полученную таблицу на экран
df_pivot3

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11407,931
не женат / не замужем,2536,274


Теперь рассчитаем, какой же процент составляют клиенты каждой категории по аналогии с прошлой гипотезой. Добавим результат в новый столбец `pivot`. Таким образом, в `pivot` хранится процент, который составляют клиенты определенной категории от общего числа клиентов

In [64]:
#считаем процент клиентов и добавляем в отдельный столбец
df_pivot3['pivot'] = ((df_pivot3[1]+df_pivot3[0])/tot_pay)*100
#выводим таблицу на экран
df_pivot3

debt,0,1,pivot
family_status,,,
в разводе,1110,85,5.570317
вдовец / вдова,896,63,4.470237
гражданский брак,3763,388,19.349275
женат / замужем,11407,931,57.511770
не женат / не замужем,2536,274,13.098401


Мы получили данные для предварительного анализа. Из полученных данных можно сделать вывод о том, что чаще всего кредит берут **женатые/замужние** граждане, за ними идут лица, состоящие **в гражданском браке**, после них идут те, кто **не женат или не замужем**, перподследнее место по количествам взятых кредитов занимают лица, находящиеся **в разводе**, и наименее заинтересованными в получении кредита лицами являются **вдовцы и вдовы**.

Если составлять рейтинг заинтересованности в получении кредита лиц, то он будет выглядеть так:
1. *женат / замужем* - 57,5%
2. *гражданский брак* - 19%
3. *не женат / не замужем* - 13%
4. *в разводе* - 5,6%
5. *вдовец / вдова* - 4,5%

Теперь добавим столбец, который содержит информацию о том, какой процент среди всех клиентов данной категории имел задолженность по кредиту, и назовем этот столбец `non-payment`. Также для более обширного анализа заодно и добавим процент тех, кто не имел задолженности и поместим результат в столбец `payment`.

In [65]:
#рассчитываем процент задолжником
df_pivot3['non-payment'] = (df_pivot3[1]/(df_pivot3[1]+df_pivot3[0]))*100
#рассчитываем процент не задолжников
df_pivot3['payment'] = (df_pivot3[0]/(df_pivot3[1]+df_pivot3[0]))*100
#выводим полученную таблицу на экран
df_pivot3

debt,0,1,pivot,non-payment,payment
family_status,,,,,
в разводе,1110,85,5.570317,7.112971,92.887029
вдовец / вдова,896,63,4.470237,6.569343,93.430657
гражданский брак,3763,388,19.349275,9.347145,90.652855
женат / замужем,11407,931,57.511770,7.545793,92.454207
не женат / не замужем,2536,274,13.098401,9.750890,90.249110


**Вывод**

Полученные результаты свидетельствуют об наличии более отличных друг от друга значений процентов просрочки, нежели те, что были рассмотрены ранее. Так процент просрочки у всех категорий колеблется в пределах от 6% до 10%.

Наиболее надежными заемщиками оказались `вдовы и вдовцы` - они просрачивали платеж лишь в **6,6%** случаев. Эта группа выделяется на оне остальных, поскольку является группой не только наименее заинтересованной в получении кредита, но и той, что реже всех остальных просрачиает платеж.

Второй по надежности является категория лиц, которые находятся `в разводе` - они просрачивали платеж в **7,1%** случаев. Эта группа является второй по отсутствию заинтересованности в кредите. Можно сказать, что самые надежные группы заемщиков - те, что наименее заинтересованы в получении кредита. 

За ними идут по проценту надежности `женатые/замужние` лица, которые просрочили платеж в **7,5%** случаев. Несмотря на большой процент получения кредитов, который мог бы послужить причиной размытия результатов, данная группа все же практически в 93 случаях из ста оплачивала кредит вовремя, что является очень хорошим показателем.

Лица же, состоящие `в гражданском браке` являются не лучшими заемщиками, поскольку процент просрочки платежей у них составляет целых **9,4%**, что на фоне предыдущих результатов заметно выделяется.

И наконец самыми ненадежными заемщиками являются `незамужние и неженатые` лица - процент просрочки платежей у данной категории составляет аж **9,8%**. Можно сказать, что практически каждый 10 неженатый или незамужний гражданин просрочил платеж по кредиту. Это довольно большой результат.

Можно проследить интересную закономерность:
- Лица, которые были женаты или замужем, но на момент получения кредита, таковыми не являлись, являются наиболее надежными заемщиками.
- Лица находящиеся в браке являются в целом в большинстве своем надежными заемщиками. Они занимают промежуточное положение между ненадежными заемщиками и надежными.
- Лица, которые еще не вступали в брак, являются наименее надежными заемщиками.

Можно составить некий рейтинг надежности заемщиков по процентам просрочки:
1. *вдовец / вдова* - 6,6%
2. *в разводе* - 7,1%
3. *женат / замужем* -7,5%
4. *гражданский брак* - 9,4%
5. *не женат / не замужем* - 9,8%

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала выведем полученные категории дохода

In [66]:
#выводим уникальные значения
df['children'].value_counts()

0    14091
1     4855
2     2127
3      330
4       41
5        9
Name: children, dtype: int64

Для работы с данными создадим сводную таблицу с помощью pivot_table, где:
- *index* - категория цели (`children`)
- *columns* - неуплата вовремя кредита (`debt`)
- *values* - значение может быть любое, поскольку количество ячеек в столбцах одинаково. (я взял за основу - `education_id`)
- *aggfunc* - функция подсчета количества (`count`)

In [67]:
#создаем сводную таблицу
df_pivot4 = df.pivot_table(index ='children', columns = 'debt', values = 'education_id', aggfunc = 'count')
#выведем полученную таблицу на экран
df_pivot4

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1925.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


В столбце, который содержит информацию о количестве просрочек клиентов с 5 детьми появилось значение NaN. Нужно выяснить, почему так произошло. Выведем на экран всех.

In [68]:
#выводим на экран клиентов с 5 детьми
df[df['children'] == 5]

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose,category_purpose,category_income
3977,5,0,42,среднее,1,гражданский брак,1,M,сотрудник,0,142,на покупку своего автомобиля,автомобиль,средний
4394,5,8,36,среднее,1,женат / замужем,0,F,компаньон,0,168,операции с недвижимостью,недвижимость,средний
7858,5,2,36,среднее,1,женат / замужем,0,F,сотрудник,0,48,операции с жильем,недвижимость,низкий
15786,5,1,31,среднее,1,женат / замужем,0,F,сотрудник,0,77,сделка с подержанным автомобилем,автомобиль,низкий
15880,5,6,37,среднее,1,женат / замужем,0,F,сотрудник,0,256,покупка недвижимости,недвижимость,высокий
16172,5,1,35,среднее,1,гражданский брак,1,F,госслужащий,0,126,на проведение свадьбы,свадьба,средний
20386,5,0,38,начальное,3,женат / замужем,0,F,сотрудник,0,212,заняться высшим образованием,образование,высокий
20769,5,6,35,среднее,1,женат / замужем,0,F,компаньон,0,204,жилье,недвижимость,высокий
21086,5,4,59,среднее,1,женат / замужем,0,M,сотрудник,0,269,операции со своей недвижимостью,недвижимость,высокий


Мы видим, что клиенты с 5 детьми ни разу не просрачивали платеж.

Проверим наше предположение:

In [69]:
#выводим на экран клиентов с 5 детьми и просрочкой платежа
df[(df['children'] == 5) & (df['debt'] == 1)]

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income(*1000),purpose,category_purpose,category_income


Предположение подтвердилось - количество клиентов с 5 детьми, которые просрочили платеж равно 0. Заменим пропуски в таблице на 0 с помощью `fillna()` и приведем значения к типу int64 - из-за появления NaN тип значений в таблице сбился.

In [70]:
#устраняем пропуски
df_pivot4 = df_pivot4.fillna(0)
#заменяем тип данных в таблице
df_pivot4 = df_pivot4.astype('int')
#выводим таблицу на экран после исправления
df_pivot4

debt,0,1
children,,
0,13028,1063
1,4410,445
2,1925,202
3,303,27
4,37,4
5,9,0


Теперь можно рассчитать, какой же процент составляют клиенты каждой категории по аналогии с прошлой гипотезой. Добавим результат в новый столбец `pivot`. Таким образом, в `pivot` хранится процент, который составляют клиенты определенной категории от общего числа клиентов

In [71]:
#считаем процент клиентов и добавляем в отдельный столбец
df_pivot4['pivot'] = ((df_pivot4[1]+df_pivot4[0])/tot_pay)*100
#выводим таблицу на экран
df_pivot4

debt,0,1,pivot
children,,,
0,13028,1063,65.683121
1,4410,445,22.630867
2,1925,202,9.914697
3,303,27,1.538246
4,37,4,0.191115
5,9,0,0.041952


Мы получили данные для предварительного анализа. Из полученных данных можно сделать вывод о том, что чаще всего кредиты берут лица, у которых нет детей, за ними следуют клиенты с **1 ребенком**, следом идут лица, у которых **2 ребенка**, меньше, чем эти лица брали кредиты те, у кого **3 ребенка**, после идут те, у кого **4 ребенка**, а наименее заинтересованными в получении кредита являются те, у кого **5 детей**.

Так, мы видим закономерность - чем больше у лиц детей, тем они меньше берут кредиты.

Если составлять рейтинг заинтересованности в получении кредита лиц, то он будет выглядеть так:
1. *нет детей* - 65,8%
2. *1 ребенок* - 22,6%
3. *2 ребенка* - 9,9%
4. *3 ребенка* - 1,5%
5. *4 ребенка* - 0,2%
6. *5 детей* - 0,04%

Теперь добавим столбец, который содержит информацию о том, какой процент среди всех клиентов данной категории имел задолженность по кредиту, и назовем этот столбец `non-payment`. Также для более обширного анализа заодно и добавим процент тех, кто не имел задолженности и поместим результат в столбец `payment`.

In [72]:
#рассчитываем процент задолжником
df_pivot4['non-payment'] = (df_pivot4[1]/(df_pivot4[1]+df_pivot4[0]))*100
#рассчитываем процент не задолжников
df_pivot4['payment'] = (df_pivot4[0]/(df_pivot4[1]+df_pivot4[0]))*100
#выводим полученную таблицу на экран
df_pivot4

debt,0,1,pivot,non-payment,payment
children,,,,,
0,13028,1063,65.683121,7.543822,92.456178
1,4410,445,22.630867,9.165808,90.834192
2,1925,202,9.914697,9.496944,90.503056
3,303,27,1.538246,8.181818,91.818182
4,37,4,0.191115,9.756098,90.243902
5,9,0,0.041952,0.000000,100.000000


**Вывод**

Гипотеза подтвердилась - факт возврата платежа в срок зависит от количества детей клиента.

Самыми надежными заемщиками являются лица с `5 детьми`. Они ни разу не просрочили платеж (**0%**) по кредиту. Можно это объяснить очень низким процессом взятия кредита лицами, принадлежащими данной группе. В частности, лица без детей брали кредит в 1,5 тысячи раз чаще, чем те, у которых 5 детей.

После клиентов с 5 детьми наиболее надежными выступают клиенты без детей. Процент просрачивания платежа в их случае составляет **7,5%**. Эти показатели кажутся очень внушительными, особенно учитывая, что это наиболее закредитованная группа населения.

Следующими по надежности выступают клиенты с `3 детьми`. Процент просрочки платежа для этих лиц составляет **8,2%**.

Менее надежными, чем выше указанные лица являются клиенты с `1 ребенком` - процент их просрочки платежа составляет **9,2%**. Это довольно высокий показатель - особенно, если сравнивать с ранее указанными результатами.

Предпоследними по надежности выступают те клиенты, у которых `2 детей`. Их процент просрочки платежа составляет **9,5%**.

Самыми же ненадежными заемщиками выступают те клиенты, у которых `4 детей`. Процент просрочки для данной категории составляет **9,8%**, что в принципе можно трактовать, как то, что каждый 10 клиент с 4 детьми просрачивает платеж.

Если составлять по процентам просрочки платежей рейтинг надежности клиентов, то он будет выглядеть так:
1. *5 детей* - 0%
2. *нет детей* - 7,5%
3. *3 ребенка* - 8,2%
4. *1 ребенок* - 9,2%
5. *2 детей* - 9,5% 
6. *4 детей* - 9,8%

## Шаг 4. Общий вывод

Мы проверили 4 гипотезы и установили:
1. Разные цели кредита влияют на его возврат в срок.
Люди, которые берут кредит на автомобиль чаще остальных не возвращают кредит в срок.
Клиенты же, которые берут кредит на недвижимость являются наиболее надежными заемщиками и чаще других возвращают кредит в срок.
`Первая гипотеза подтвердилась полностью.`


2. Зависимость между уровнем дохода и возвратом кредита в срок сущетсвует, но она крайне мала.
Данная зависимость заметна только для случаев, когда мы сравниваем клиентов с очень высокими доходами, явно отличающимися от остальных, и других клиентов со средними и низкими доходами.
`Вторая гипотеза подтвердилась частично.`


3. Существует зависимость между семейным положением и возвратом кредита в срок
Если клиент находился или находится в браке, то он является более надежным заемщиком и чаще возвращал кредит в срок, нежели тото, кто еще не вступал в брак.
`Третья гипотеза подтвердилась полностью.`


4. Зависимость между наличием детей и возвратом кредита в срок существует и она является довольно яркой и заметной.
Наиболее надежными заемщиками являются две категории клиентов - те, у кого нет детей, и те, у кого 5 детей. Данные клиенты чаще остальных возвращали кредит в срок, а клиенты с 5 детьми еще ни разу не просрачивали платеж.
Самые же ненадежные заемщики - это клиенты с 4 детьми.
`Четвертая гипотеза подтвердилась полностью.`